In [18]:
import random
import numpy as np

color = [
    ['A', 'B', 'B', 'C', 'C', 'C'],
    ['A', 'B', 'B', 'C', 'C', 'C'],
    ['A', 'A', 'B', 'B', 'C', 'C'],
    ['A', 'A', 'B', 'B', 'C', 'C'],
    ['A', 'A', 'A', 'B', 'B', 'C'],
    ['A', 'A', 'A', 'B', 'B', 'C'],
]

def get_neighbors(cell):
    x, y = cell
    
    neighbors = []
    
    for dx in [-1,1,-2,2]:
        for dy in [-1,1,-2,2]:
            if abs(dx) == abs(dy):
                continue
            if x + dx < 0 or x + dx >= len(color) or y + dy < 0 or y + dy >= len(color):
                continue
            neighbors += [(x+dx, y+dy)]
    
    return neighbors

In [19]:
def evaluate_placement_path(A, B, C, path):
    ans = A
    for i, cell in enumerate(path[1:]):
        prev_cell = path[i]
        if color[cell[0]][cell[1]] == color[prev_cell[0]][prev_cell[1]]:
            if color[cell[0]][cell[1]] == 'A':
                ans += A
            elif color[cell[0]][cell[1]] == 'B':
                ans += B
            else:
                ans += C
        
        else:
            if color[cell[0]][cell[1]] == 'A':
                ans *= A
            elif color[cell[0]][cell[1]] == 'B':
                ans *= B
            else:
                ans *= C
    return ans

def evaluate_placement_coloring(A,B,C, coloring):
    ans = A
    for i, cur_color in enumerate(coloring[1:]):
        prev_color = coloring[i]
        if prev_color == cur_color:
            if cur_color == 'A':
                ans += A
            elif cur_color == 'B':
                ans += B
            else:
                ans += C
        
        else:
            if cur_color == 'A':
                ans *= A
            elif cur_color == 'B':
                ans *= B
            else:
                ans *= C
    return ans

In [20]:
def find_alternating_path(A,B,C, score, path, ans_ref):
    if score == 2024:
        if path[-1] == 'C' and 'B' in path:
            ans_ref += [path]
        return
    elif score > 2024 or len(path) > 15:
        return 
    
    prev = path[-1]
    
    nxt_opts = ['A', 'B', 'C']
    random.shuffle(nxt_opts)
    for nxt in nxt_opts:
        cur_score = score
        if nxt == prev:
           if nxt == 'A':
               cur_score += A
           elif nxt == 'B':
               cur_score += B
           else:
               cur_score += C
        else:
            if nxt == 'A':
                cur_score *= A
            elif nxt == 'B':
                cur_score *= B
            else:
                cur_score *= C
        find_alternating_path(A,B,C,cur_score, path + [nxt], ans_ref)
    

In [21]:
good_coloring_seq = []

cnt_printed = 0

for A in range(1,50):
    for B in range(1,50):
        for C in range(1,50):
            
            if A == B or B == C or A == C:
                continue
            if A + B + C != 6: # not an optimal A + B + C
                continue
                
            cur = []
            find_alternating_path(A,B,C, A, ['A'], cur)
            for path in cur:
                good_coloring_seq += [((A, B, C), path, )]
                
                if cnt_printed <= 50:
                    print(good_coloring_seq[-1])
                    cnt_printed += 1

((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'C', 'B', 'C', 'C', 'C', 'B', 'A', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'C', 'B', 'C', 'C', 'C', 'B', 'A', 'A', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'C', 'B', 'A', 'A', 'B', 'B', 'C', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'C', 'B', 'A', 'A', 'A', 'B', 'C', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'C', 'B', 'A', 'B', 'B', 'B', 'C', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'A', 'B', 'C', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'B', 'C', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'B', 'C', 'A', 'B', 'B', 'B', 'C', 'B', 'B', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'A', 'C', 'B', 'B', 'A', 'C', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'A', 'C', 'B', 'B', 'A', 'A', 'C'])
((1, 3, 2), ['A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'A

In [22]:
def find_path_with_coloring(cur_cell, target_cell, path, coloring):
    
    path = path.copy() + [cur_cell]
    if cur_cell == target_cell and len(path) == len(coloring):
        # print(path)
        return path
    
    if len(path) == len(coloring):
        return []
    
    neighbors = get_neighbors(cur_cell)
    random.shuffle(neighbors)
    
    next_color = coloring[len(path)]
    
    for neighbor in neighbors:
        if color[neighbor[0]][neighbor[1]] == next_color and neighbor not in path:
            ans = find_path_with_coloring(neighbor, target_cell, path, coloring)
            if ans:
                return ans

In [26]:
values_2_colorings = {}

for values, coloring in good_coloring_seq:
    if values not in values_2_colorings:
        values_2_colorings[values] = []
    values_2_colorings[values] += [coloring]

In [27]:
all_colorings = []
for coloring in good_coloring_seq:
    all_colorings.append(coloring[1])

In [28]:
for values, colorings in values_2_colorings.items():
    if np.sum(values) > 6:
        continue
    for coloring in colorings:
        
        forward_path = find_path_with_coloring((0,0), (5,5), [], coloring)
        backward_path = find_path_with_coloring((5,0), (0,5), [], coloring)
        if forward_path and backward_path:
            print(f'found {(values, forward_path, backward_path)=}')


found (values, forward_path, backward_path)=((1, 3, 2), [(0, 0), (2, 1), (4, 0), (3, 2), (5, 1), (4, 3), (3, 5), (1, 4), (2, 2), (0, 3), (1, 1), (2, 3), (4, 2), (3, 4), (5, 5)], [(5, 0), (4, 2), (3, 0), (2, 2), (1, 0), (0, 2), (1, 4), (3, 5), (4, 3), (2, 4), (1, 2), (3, 3), (2, 1), (1, 3), (0, 5)])
found (values, forward_path, backward_path)=((1, 3, 2), [(0, 0), (2, 1), (4, 2), (2, 3), (3, 1), (1, 2), (2, 4), (4, 3), (2, 2), (0, 1), (1, 3), (3, 2), (5, 3), (3, 4), (5, 5)], [(5, 0), (3, 1), (1, 0), (0, 2), (2, 1), (3, 3), (2, 5), (4, 4), (2, 3), (1, 1), (0, 3), (2, 2), (4, 3), (2, 4), (0, 5)])
found (values, forward_path, backward_path)=((1, 3, 2), [(0, 0), (2, 1), (4, 0), (3, 2), (1, 3), (0, 1), (2, 0), (1, 2), (3, 3), (5, 4), (3, 5), (4, 3), (2, 2), (3, 4), (5, 5)], [(5, 0), (3, 1), (1, 0), (0, 2), (1, 4), (3, 3), (4, 1), (5, 3), (3, 2), (1, 1), (0, 3), (2, 2), (4, 3), (2, 4), (0, 5)])
found (values, forward_path, backward_path)=((1, 3, 2), [(0, 0), (2, 1), (4, 2), (2, 3), (3, 5), (4,

In [31]:
(A, B, C), forward_path, backward_path = ((1, 3, 2), [(0, 0), (2, 1), (4, 2), (2, 3), (3, 1), (4, 3), (3, 5), (5, 4), (3, 3), (1, 2), (2, 4), (3, 2), (5, 3), (3, 4), (5, 5)], [(5, 0), (4, 2), (2, 1), (0, 2), (1, 0), (2, 2), (3, 4), (5, 3), (3, 2), (4, 4), (2, 5), (3, 3), (1, 2), (2, 4), (0, 5)])


print(f'forward path score: {evaluate_placement_path(A, B, C, forward_path)}')
print(f'backward path score: {evaluate_placement_path(A, B, C, backward_path)}')

forward path score: 2024
backward path score: 2024
